# Lab 4
## You are tasked with evaluating card counting strategies for black jack. In order to do so, you will use object oriented programming to create a playable casino style black jack game where a computer dealer plays against n computer players and possibily one human player. If you don't know the rules of blackjack or card counting, please google it.

## A few requirements:

* The game should utilize multiple 52-card decks. Typically the game is played with 6 decks.
* Players should have chips.
* Dealer's actions are predefined by rules of the game (typically hit on 16).
* The players should be aware of all shown cards so that they can count cards.
* Each player could have a different strategy.
* The system should allow you to play large numbers of games, study the outcomes, and compare average winnings per hand rate for different strategies.

## 1. Begin by creating a classes to represent cards and decks. The deck should support more than one 52-card set. The deck should allow you to shuffle and draw cards. Include a "plastic" card, placed randomly in the deck. Later, when the plastic card is dealt, shuffle the cards before the next deal.

In [24]:
import random
import numpy as np
import matplotlib.pyplot as plt

In [25]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
        self.value = self.card_value()
    def card_value(self):
        if self.rank in ['J', 'Q', 'K']:
            return 10
        elif self.rank == 'A':
            return 11
        else:
            return int(self.rank)
    def __str__(self):
        return f"{self.rank} of {self.suit}"
    
                
class Deck:

    def __init__(self, num_decks=6):
        self.num_decks = num_decks
        self.cards = []
        self.deck()
    def deck(self):
        suits=["Heart", "Spades", "Diamonds", "Clubs"]
        ranks=["2","3","4","5","6","7","8","9","10","J","Q","K","A"]
        self.cards = [Card(suit,rank) for i in range(self.num_decks) for suit in suits for rank in ranks]
    def shuffle(self):
        random.shuffle(self.cards) 
    def draw_card(self):
        #Return a singular card
        drawn_deck = self.cards.pop(0)
        return drawn_deck
    def plastic(self):
        plastic_index = -1
        for i, card in enumerate(self.cards):
            if card.rank == "Plastic":
                plastic_index = i
                break
        current_index = len(self.cards) - 1
        return plastic_index <= current_index

In [26]:
deck = Deck()
for i in range(52):
    card = deck.draw_card()
    print(card)
should_shuffle = deck.plastic()
print("Should shuffle:", should_shuffle)

2 of Heart
3 of Heart
4 of Heart
5 of Heart
6 of Heart
7 of Heart
8 of Heart
9 of Heart
10 of Heart
J of Heart
Q of Heart
K of Heart
A of Heart
2 of Spades
3 of Spades
4 of Spades
5 of Spades
6 of Spades
7 of Spades
8 of Spades
9 of Spades
10 of Spades
J of Spades
Q of Spades
K of Spades
A of Spades
2 of Diamonds
3 of Diamonds
4 of Diamonds
5 of Diamonds
6 of Diamonds
7 of Diamonds
8 of Diamonds
9 of Diamonds
10 of Diamonds
J of Diamonds
Q of Diamonds
K of Diamonds
A of Diamonds
2 of Clubs
3 of Clubs
4 of Clubs
5 of Clubs
6 of Clubs
7 of Clubs
8 of Clubs
9 of Clubs
10 of Clubs
J of Clubs
Q of Clubs
K of Clubs
A of Clubs
Should shuffle: True


## 2. Now design your game on a UML diagram. You may want to create classes to represent, players, a hand, and/or the game. As you work through the lab, update your UML diagram. At the end of the lab, submit your diagram (as pdf file) along with your notebook.

## 3. Begin with implementing the skeleton (ie define data members and methods/functions, but do not code the logic) of the classes in your UML diagram.


## 4. Complete the implementation by coding the logic of all functions. For now, just implement the dealer player and human player.

In [27]:
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        self.aces = 0
    def __iter__(self):
        self.current = 0
        return self
    def __next__(self):
        if self.current < len(self.cards):
            card = self.cards[self.current]
            self.current += 1
            return card
        else:
            raise StopIteration
    def add_card(self, card):
        self.cards.append(card)
        self.value += card.value
        
        if card.rank == 'A':
            self.aces += 1        
    def ace_adjust(self):
        while self.value > 21 and self.aces:
            self.value -= 10
            self.aces -= 1          
    def get_cards(self, show_first=True):
        cards = []
        for i, card in enumerate(self.cards):
            if i == 0 and not show_first:
                cards.append('Hidden Card')
            else:
                cards.append(str(card))
        return cards
    def clear_hand(self):
        self.hand = Hand()
    def __iter__(self):
        return iter(self.cards)
    def get_value(self):
        return self.value
    def is_bust(self):
        return self.value>21
    def should_hit(self):
        return self.value < 17

In [28]:
class Participant:
    def __init__(self):
        self.hand = Hand()
    def add_card(self, card,hidden=False):
        self.hand.add_card(card)
        self.hand.ace_adjust()  
    def show_hand(self, show_first=True):
        cards = []
        for i, card in enumerate(self.hand.get_cards(show_first)):
            if i == 0 and not show_first:
                cards.append('Hidden Card')
            else:
                cards.append(str(card))
        return cards        
    def clear_hand(self):
        self.hand = Hand()
    def hit(self, deck):
        self.add_card(deck.draw_card())


In [29]:

class Chips:
    def __init__(self, total=1000):
        self.total = total
        self.bet = 0
    def win_bet(self):
        self.total += self.bet
        self.bet = 0
    def lose_bet(self):
        self.bet = 0
    def blackjack_win(self):
        self.total += int(self.bet * 1.5)
        self.bet = 0
        return self.total
    def take_bet(self, amount):
        if amount > self.total:
            print("Sorry, you don't have enough chips!")
            return False
        else:
            self.bet = amount
            self.total -= amount
            return True


In [30]:
class Player(Participant):

    def __init__(self,name):
        super().__init__()
        self.chips = Chips()
        self.bet_won = False
        self.name = name
        
    def hit(self, deck):
        #deck is an instance of the Deck class.
        self.add_card(deck.draw_card())
        
    def stand(self):
        pass
    
    def lose_bet(self):
        self.chips.lose_bet()

    def bet(self, chips):
        self.bet_won = self.chips.take_bet(chips)
    
    def bet_winner(self):
        if self.bet_won==True:
            self.chips.win_bet()
            print(f"{self.name} have won {payout} in chips!")
        else:
            print("Sorry, you didn't win the previous bet.")
            
    def blackjack_win(self):
        # if player gets blackjack, add the winnings to the player's chip total.
        payout=self.chips.blackjack_win()
        print(f"Player {self.name} won {payout} chips with a blackjack!")
        
    def show_hand(self):
        print(f"{self.name}'s hand: {' | '.join(self.hand.get_cards())}")

    def get_score(self):
        score = self.hand.get_value()
        return score
    def is_bust(self):
        return self.value>21

In [31]:
class Dealer(Participant):
   
    def __init__(self):
        super().__init__()
    def play(self, deck):
        while self.hand_value < 17:
            self.add_card(deck.draw_card())
        if self.hand_value > 21:
            print("Dealer busted!")
        else:
            print(f"Dealer stands on {self.hand_value}.") 
    def hit(self, deck):
        self.add_card(deck.draw_card())
    def show_hand(self, show_first=True):
        if show_first:
            print(f"Dealer's hand: {str(self.hand.cards[0])}, ***")
        else:
            print(f"Dealer's hand: {self.hand}")
    def get_score(self):
        score = self.hand.get_value()
        return score

## 5. Test. Demonstrate game play. For example, create a game of several dealer players and show that the game is functional through several rounds.


In [48]:
class Game:
    def __init__(self):
        self.deck = Deck()
        self.deck.shuffle()
        self.dealer = Dealer()
        self.players = []
    def add_player(self, name):
        player = Player(name)
        self.players.append(player)
    def deal(self):
        for _ in range(2):
            for player in self.players:
                player.hit(self.deck)
            self.dealer.hit(self.deck)
        
        self.dealer.show_hand()
        for player in self.players:
            player.show_hand()   
    def play(self):
        self.deal()
        for player in self.players:
            while True:
                print(f"{player.name}'s turn.")
                print("Current hand:")
                player.show_hand()
                choice = input("Do you want to hit or stand? ").lower()
                if choice == "hit":
                    player.hit(self.deck)
                    if player.hand.is_bust():
                        print("Bust!")
                        break
                elif choice == "stand":
                    player.stand()
                    break
        self.dealer.show_hand()
        while self.dealer.hand.should_hit():
            self.dealer.hit(self.deck)
        self.winner()
    def winner(self):
        dealer_score = self.dealer.get_score()
        for player in self.players:
            player_score = player.get_score()
        if player_score > 21:
            print(f"{player.name} busts with {player_score} points!")
        elif dealer_score > 21:
            print(f"{player.name} wins with {player_score} points! Dealer busts with {dealer_score} points!")
            player.bet_winner()
        elif player_score > dealer_score:
            print(f"{player.name} wins with {player_score} points! Dealer has {dealer_score} points.")
            player.bet_winner()
        elif dealer_score > player_score:
            print(f"Dealer wins with {dealer_score} points! {player.name} has {player_score} points.")
        else:
            print(f"{player.name} ties with the dealer with {player_score} points.")
            player.chips.return_bet()
    def play_game(self, num_rounds=50):
        for i in range(num_rounds):
            print(f"\n--- Round {i+1} ---")
            self.play_round()
            self.rounds_played += 1

In [49]:
game = Game()
game.add_player("Alyssa")
game.add_player("James")
game.deal()
game.play()


Dealer's hand: 10 of Clubs, ***
Alyssa's hand: Q of Diamonds | 8 of Heart
James's hand: 6 of Spades | 3 of Diamonds
Dealer's hand: 10 of Clubs, ***
Alyssa's hand: Q of Diamonds | 8 of Heart | 4 of Heart | 7 of Diamonds
James's hand: 6 of Spades | 3 of Diamonds | 7 of Diamonds | Q of Spades
Alyssa's turn.
Current hand:
Alyssa's hand: Q of Diamonds | 8 of Heart | 4 of Heart | 7 of Diamonds
Do you want to hit or stand? hit
Bust!
James's turn.
Current hand:
James's hand: 6 of Spades | 3 of Diamonds | 7 of Diamonds | Q of Spades
Do you want to hit or stand? hit
Bust!
Dealer's hand: 10 of Clubs, ***
James busts with 36 points!


## 6.Implement a new player with the following strategy:

* Assign each card a value:
    * Cards 2 to 6 are +1
    * Cards 7 to 9 are 0
* Cards 10 through Ace are -1
* Compute the sum of the values for all cards seen so far.
* Hit if sum is very negative, stay if sum is very positive. Select a threshold for hit/stay, e.g. 0 or -2.

Player should stand


## 7. Create a test scenario where one player, using the above strategy, is playing with a dealer and 3 other players that follow the dealer's strategy. Each player starts with same number of chips. Play 50 rounds (or until the strategy player is out of money). Compute the strategy player's winnings. You may remove unnecessary printouts from your code (perhaps implement a verbose/quiet mode) to reduce the output.

## 8. Create a loop that runs 100 games of 50 rounds, as setup in previous question, and store the strategy player's chips at the end of the game (aka "winnings") in a list. Histogram the winnings. What is the average winnings per round? What is the standard deviation. What is the probabilty of net winning or lossing after 50 rounds?

## 9. Repeat previous questions scanning the value of the threshold. Try at least 5 different threshold values. Can you find an optimal value?


In [ ]:
N/A

## 10. Create a new strategy based on web searches or your own ideas. Demonstrate that the new strategy will result in increased or decreased winnings.

In [ ]:
N/A